In [1]:
import pandas as pd
import numpy as np
import os
import pickle

def summarize_metrics(path, show_train=True, show_test=True, show_scalar=True, runs=11):
    results = pd.read_pickle(path)

    train_metric_sums = {}
    test_metric_sums = {}
    scalar_sums = {
        'training_time_sec': 0.0,
        'gpu_mem_MB': 0.0,
        'cpu_mem_MB': 0.0,
        'param_count': 0.0
    }

    count = 0
    for run in range(runs):
        if run not in results:
            continue
        entry = results[run]
        count += 1

        for key in scalar_sums:
            scalar_sums[key] += entry.get(key, 0.0)

        for k, v in entry.get('train_metrics', {}).items():
            train_metric_sums[k] = train_metric_sums.get(k, 0.0) + float(v)

        for k, v in entry.get('test_metrics', {}).items():
            test_metric_sums[k] = test_metric_sums.get(k, 0.0) + float(v)

    # 평균 계산
    scalar_avg = {f"avg_{k}": round(v / count, 2) for k, v in scalar_sums.items()} if show_scalar else {}
    train_avg = {f"train_{k}": round(v / count, 2) for k, v in train_metric_sums.items()} if show_train else {}
    test_avg = {f"test_{k}": round(v / count, 2) for k, v in test_metric_sums.items()} if show_test else {}

    # 통합 및 DataFrame 변환
    all_metrics = {**scalar_avg, **train_avg, **test_avg}
    df = pd.DataFrame([all_metrics])

    return df

dataset_order = [
    "basic", "gaussian", "uniform", "outlier", "edge", "BA", "ER", "grid", "tree",
    "education", "election", "income", "unemployment",
    "PTBR", "chameleon", "crocodile", "squirrel", "Anaheim", "ChicagoSketch"
]

def get_dataset_index(filename):
    for i, name in enumerate(dataset_order):
        if name in filename:
            return i
    return len(dataset_order)  # 못 찾은 건 맨 뒤로

def summarize_model_resources(path, runs=11):
    results = pd.read_pickle(path)

    models = ['gnn', 'conf_gnn']
    scalar_keys = ['training_time_sec', 'gpu_mem_MB', 'cpu_mem_MB', 'param_count']

    # 모델별 누적 딕셔너리 초기화
    model_sums = {model: {k: 0.0 for k in scalar_keys} for model in models}
    model_counts = {model: 0 for model in models}

    for run in range(runs):
        if run not in results:
            continue
        for model in models:
            entry = results[run].get(model, {})
            if not entry:
                continue
            model_counts[model] += 1
            for key in scalar_keys:
                model_sums[model][key] += entry.get(key, 0.0)

    # 평균 계산
    model_avgs = {}
    for model in models:
        if model_counts[model] == 0:
            continue
        avg_metrics = {
            f"{model}_{k}": round(v / model_counts[model], 4)
            for k, v in model_sums[model].items()
        }
        model_avgs.update(avg_metrics)

    df = pd.DataFrame([model_avgs])
    return df

## Alone Model

### SQR

In [79]:
train, test, scalar = False, True, False

folder_path = "/home/sypark/GQNN/pred (0401)/SQR"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
sorted_pkl_files = sorted(pkl_files, key=get_dataset_index)

In [4]:
for i in range(len(pkl_files)):
    print('SQR', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).iloc[:, 3:5].T)
    print('-' * 30)


SQR basic_SQR.pkl Results...
             0
test_MCT  0.02
test_CWC  1.01
------------------------------
SQR gaussian_SQR.pkl Results...
             0
test_MCT  0.01
test_CWC  1.32
------------------------------
SQR uniform_SQR.pkl Results...
             0
test_MCT  0.02
test_CWC  1.27
------------------------------
SQR outlier_SQR.pkl Results...
            0
test_MCT  0.0
test_CWC  0.2
------------------------------
SQR edge_SQR.pkl Results...
             0
test_MCT  0.01
test_CWC  0.64
------------------------------
SQR BA_SQR.pkl Results...
             0
test_MCT  0.07
test_CWC  3.20
------------------------------
SQR ER_SQR.pkl Results...
             0
test_MCT  0.08
test_CWC  4.27
------------------------------
SQR grid_SQR.pkl Results...
             0
test_MCT  0.06
test_CWC  2.87
------------------------------
SQR tree_SQR.pkl Results...
             0
test_MCT  0.02
test_CWC  1.38
------------------------------
SQR education_SQR.pkl Results...
             0
test_MCT  0.

In [80]:
for i in range(len(pkl_files)):
    print('SQR', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).iloc[:, 5:].T)
    print('-' * 30)

SQR basic_SQR.pkl Results...
                   0
test_MPE        0.09
test_Sharpness  0.14
test_WS         0.33
------------------------------
SQR gaussian_SQR.pkl Results...
                   0
test_MPE        0.13
test_Sharpness  0.26
test_WS         0.51
------------------------------
SQR uniform_SQR.pkl Results...
                   0
test_MPE        0.13
test_Sharpness  0.27
test_WS         0.52
------------------------------
SQR outlier_SQR.pkl Results...
                   0
test_MPE        0.05
test_Sharpness  0.01
test_WS         0.12
------------------------------
SQR edge_SQR.pkl Results...
                   0
test_MPE        0.08
test_Sharpness  0.13
test_WS         0.32
------------------------------
SQR BA_SQR.pkl Results...
                   0
test_MPE        0.25
test_Sharpness  0.54
test_WS         0.73
------------------------------
SQR ER_SQR.pkl Results...
                   0
test_MPE        0.21
test_Sharpness  0.38
test_WS         0.61
-----------------------

### RQR

In [81]:
train, test, scalar = False, True, False

folder_path = "/home/sypark/GQNN/pred (0401)/RQR"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
sorted_pkl_files = sorted(pkl_files, key=get_dataset_index)

In [83]:
for i in range(len(pkl_files)):
    print('RQR', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).iloc[:, 3:5].T) 
    print('-' * 30)

RQR basic_RQR.pkl Results...
             0
test_MCT  0.00
test_CWC  1.66
------------------------------
RQR gaussian_RQR.pkl Results...
             0
test_MCT  0.01
test_CWC  1.34
------------------------------
RQR uniform_RQR.pkl Results...
             0
test_MCT  0.00
test_CWC  1.48
------------------------------
RQR outlier_RQR.pkl Results...
             0
test_MCT  0.00
test_CWC  0.71
------------------------------
RQR edge_RQR.pkl Results...
             0
test_MCT  0.03
test_CWC  1.43
------------------------------
RQR BA_RQR.pkl Results...
             0
test_MCT  0.09
test_CWC  3.41
------------------------------
RQR ER_RQR.pkl Results...
             0
test_MCT  0.01
test_CWC  1.77
------------------------------
RQR grid_RQR.pkl Results...
             0
test_MCT  0.09
test_CWC  3.87
------------------------------
RQR tree_RQR.pkl Results...
             0
test_MCT  0.04
test_CWC  2.14
------------------------------
RQR education_RQR.pkl Results...
             0
test_MCT 

In [82]:
for i in range(len(pkl_files)):
    print('RQR', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).iloc[:, 5:].T) 
    print('-' * 30)

RQR basic_RQR.pkl Results...
                   0
test_MPE        0.25
test_Sharpness  0.67
test_WS         0.82
------------------------------
RQR gaussian_RQR.pkl Results...
                   0
test_MPE        0.14
test_Sharpness  0.29
test_WS         0.54
------------------------------
RQR uniform_RQR.pkl Results...
                   0
test_MPE        0.17
test_Sharpness  0.46
test_WS         0.68
------------------------------
RQR outlier_RQR.pkl Results...
                   0
test_MPE        0.11
test_Sharpness  0.13
test_WS         0.37
------------------------------
RQR edge_RQR.pkl Results...
                   0
test_MPE        0.25
test_Sharpness  0.68
test_WS         0.83
------------------------------
RQR BA_RQR.pkl Results...
                   0
test_MPE        0.28
test_Sharpness  0.57
test_WS         0.76
------------------------------
RQR ER_RQR.pkl Results...
                   0
test_MPE        0.25
test_Sharpness  0.60
test_WS         0.78
-----------------------

### BNN

In [16]:
train, test, scalar = False, True, False

folder_path = "/home/sypark/GQNN/pred (0401)/BNN"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
sorted_pkl_files = sorted(pkl_files, key=get_dataset_index)

In [18]:
for i in range(len(pkl_files)):
    print('BNN', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).iloc[:, 3:5].T) 
    print('-' * 30)

BNN basic_BNN.pkl Results...
             0
test_MCT  0.30
test_CWC  4.15
------------------------------
BNN gaussian_BNN.pkl Results...
             0
test_MCT  0.30
test_CWC  4.63
------------------------------
BNN uniform_BNN.pkl Results...
             0
test_MCT  0.30
test_CWC  4.39
------------------------------
BNN outlier_BNN.pkl Results...
             0
test_MCT  0.29
test_CWC  3.94
------------------------------
BNN edge_BNN.pkl Results...
             0
test_MCT  0.31
test_CWC  4.22
------------------------------
BNN BA_BNN.pkl Results...
             0
test_MCT  0.31
test_CWC  4.43
------------------------------
BNN ER_BNN.pkl Results...
             0
test_MCT  0.30
test_CWC  4.34
------------------------------
BNN grid_BNN.pkl Results...
             0
test_MCT  0.30
test_CWC  4.53
------------------------------
BNN tree_BNN.pkl Results...
             0
test_MCT  0.30
test_CWC  4.79
------------------------------
BNN education_BNN.pkl Results...
             0
test_MCT 

In [19]:
for i in range(len(pkl_files)):
    print('BNN', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).iloc[:, 5:].T) 
    print('-' * 30)

BNN basic_BNN.pkl Results...
                   0
test_MPE        0.39
test_Sharpness  9.08
test_WS         3.01
------------------------------
BNN gaussian_BNN.pkl Results...
                   0
test_MPE        0.31
test_Sharpness  8.92
test_WS         2.98
------------------------------
BNN uniform_BNN.pkl Results...
                   0
test_MPE        0.35
test_Sharpness  9.02
test_WS         3.00
------------------------------
BNN outlier_BNN.pkl Results...
                   0
test_MPE        0.09
test_Sharpness  8.72
test_WS         2.95
------------------------------
BNN edge_BNN.pkl Results...
                   0
test_MPE        0.42
test_Sharpness  9.37
test_WS         3.06
------------------------------
BNN BA_BNN.pkl Results...
                   0
test_MPE        0.40
test_Sharpness  9.50
test_WS         3.08
------------------------------
BNN ER_BNN.pkl Results...
                   0
test_MPE        0.28
test_Sharpness  9.09
test_WS         3.01
-----------------------

### MC

In [20]:
train, test, scalar = False, True, False

folder_path = "/home/sypark/GQNN/pred (0401)/MC"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
sorted_pkl_files = sorted(pkl_files, key=get_dataset_index)

In [21]:
for i in range(len(pkl_files)):
    print('MC', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).iloc[:, 3:5].T) 
    print('-' * 30)

MC basic_MC.pkl Results...
              0
test_MCT   0.17
test_CWC  71.71
------------------------------
MC gaussian_MC.pkl Results...
             0
test_MCT   0.1
test_CWC  34.7
------------------------------
MC uniform_MC.pkl Results...
              0
test_MCT   0.14
test_CWC  60.18
------------------------------
MC outlier_MC.pkl Results...
             0
test_MCT  0.03
test_CWC  8.18
------------------------------
MC edge_MC.pkl Results...
              0
test_MCT   0.16
test_CWC  58.17
------------------------------
MC BA_MC.pkl Results...
              0
test_MCT   0.13
test_CWC  47.61
------------------------------
MC ER_MC.pkl Results...
               0
test_MCT    0.15
test_CWC  157.92
------------------------------
MC grid_MC.pkl Results...
              0
test_MCT   0.09
test_CWC  55.25
------------------------------
MC tree_MC.pkl Results...
              0
test_MCT   0.12
test_CWC  85.17
------------------------------
MC education_MC.pkl Results...
              0
test

In [22]:
for i in range(len(pkl_files)):
    print('MC', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).iloc[:, 5:].T) 
    print('-' * 30)

MC basic_MC.pkl Results...
                   0
test_MPE        0.28
test_Sharpness  0.10
test_WS         0.47
------------------------------
MC gaussian_MC.pkl Results...
                   0
test_MPE        0.15
test_Sharpness  0.04
test_WS         0.27
------------------------------
MC uniform_MC.pkl Results...
                   0
test_MPE        0.21
test_Sharpness  0.07
test_WS         0.36
------------------------------
MC outlier_MC.pkl Results...
                   0
test_MPE        0.07
test_Sharpness  0.00
test_WS         0.11
------------------------------
MC edge_MC.pkl Results...
                   0
test_MPE        0.27
test_Sharpness  0.09
test_WS         0.45
------------------------------
MC BA_MC.pkl Results...
                   0
test_MPE        0.24
test_Sharpness  0.07
test_WS         0.41
------------------------------
MC ER_MC.pkl Results...
                   0
test_MPE        0.28
test_Sharpness  0.07
test_WS         0.41
------------------------------
MC gri

### CF-GNN

In [29]:
train, test, scalar = False, True, False

folder_path = "/home/sypark/GQNN/pred (0401)/CF-GNN (opt)"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
sorted_pkl_files = sorted(pkl_files, key=get_dataset_index)

In [26]:
for i in range(len(pkl_files)):
    print('CF-GNN', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).iloc[:, 3:5].T) 
    print('-' * 30)

CF-GNN optimal_education_GraphSAGE_conftr_calib_holdout_confgnn.pkl Results...
             0
test_MCT  0.05
test_CWC  0.98
------------------------------
CF-GNN optimal_election_GraphSAGE_conftr_calib_holdout_confgnn.pkl Results...
             0
test_MCT  0.02
test_CWC  1.03
------------------------------
CF-GNN optimal_income_GraphSAGE_conftr_calib_holdout_confgnn.pkl Results...
             0
test_MCT  0.06
test_CWC  0.81
------------------------------
CF-GNN optimal_unemployment_GraphSAGE_conftr_calib_holdout_confgnn.pkl Results...
             0
test_MCT  0.04
test_CWC  0.73
------------------------------
CF-GNN optimal_PTBR_GraphSAGE_conftr_calib_holdout_confgnn.pkl Results...
             0
test_MCT  0.04
test_CWC  0.75
------------------------------
CF-GNN optimal_Anaheim_GraphSAGE_conftr_calib_holdout_confgnn.pkl Results...
             0
test_MCT  0.09
test_CWC  1.37
------------------------------
CF-GNN optimal_ChicagoSketch_GraphSAGE_conftr_calib_holdout_confgnn.pkl Result

In [30]:
for i in range(len(pkl_files)):
    print('CF-GNN', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).iloc[:, 5:].T) 
    print('-' * 30)

CF-GNN optimal_education_GraphSAGE_conftr_calib_holdout_confgnn.pkl Results...
                    0
test_MPE         0.76
test_Sharpness  11.82
test_WS          3.14
------------------------------
CF-GNN optimal_election_GraphSAGE_conftr_calib_holdout_confgnn.pkl Results...
                   0
test_MPE        0.22
test_Sharpness  0.90
test_WS         0.96
------------------------------
CF-GNN optimal_income_GraphSAGE_conftr_calib_holdout_confgnn.pkl Results...
                   0
test_MPE        0.71
test_Sharpness  8.63
test_WS         2.97
------------------------------
CF-GNN optimal_unemployment_GraphSAGE_conftr_calib_holdout_confgnn.pkl Results...
                   0
test_MPE        0.67
test_Sharpness  6.88
test_WS         2.69
------------------------------
CF-GNN optimal_PTBR_GraphSAGE_conftr_calib_holdout_confgnn.pkl Results...
                   0
test_MPE        0.57
test_Sharpness  5.58
test_WS         2.38
------------------------------
CF-GNN optimal_Anaheim_GraphSAGE

### GQNN

In [45]:
train, test, scalar = False, True, False

folder_path = "/home/sypark/GQNN/pred (0401)/GQNN (opt)"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
sorted_pkl_files = sorted(pkl_files, key=get_dataset_index)

In [46]:
for i in range(len(pkl_files)):
    print('GQNN', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).iloc[:, :2].T) 
    print('-' * 30)

GQNN basic_GQNN_lf(0.0191600441967502).pkl Results...
              0
test_PICP  0.93
test_MPIW  0.72
------------------------------
GQNN gaussian_GQNN_lf(0.0191600441967502).pkl Results...
              0
test_PICP  0.96
test_MPIW  0.61
------------------------------
GQNN uniform_GQNN_lf(0.0261667441840123).pkl Results...
              0
test_PICP  0.92
test_MPIW  0.46
------------------------------
GQNN outlier_GQNN_lf(0.0071819706110315).pkl Results...
              0
test_PICP  0.99
test_MPIW  0.76
------------------------------
GQNN edge_GQNN_lf(0.0191600441967502).pkl Results...
              0
test_PICP  0.96
test_MPIW  0.74
------------------------------
GQNN BA_GQNN_lf(0.0451374425325055).pkl Results...
              0
test_PICP  0.95
test_MPIW  0.47
------------------------------
GQNN ER_GQNN_lf(0.0318481619672583).pkl Results...
              0
test_PICP  0.92
test_MPIW  0.36
------------------------------
GQNN grid_GQNN_lf(0.029537643054718).pkl Results...
              0
t

In [39]:
for i in range(len(pkl_files)):
    print('GQNN', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).iloc[:, 5:].T)
    print('-' * 30)

GQNN basic_GQNN_lf(0.01).pkl Results...
                   0
test_MPE        0.25
test_Sharpness  0.77
test_WS         0.88
------------------------------
GQNN gaussian_GQNN_lf(0.01).pkl Results...
                   0
test_MPE        0.14
test_Sharpness  0.57
test_WS         0.76
------------------------------
GQNN uniform_GQNN_lf(0.01).pkl Results...
                   0
test_MPE        0.18
test_Sharpness  0.66
test_WS         0.81
------------------------------
GQNN outlier_GQNN_lf(0.01).pkl Results...
                   0
test_MPE        0.26
test_Sharpness  0.50
test_WS         0.71
------------------------------
GQNN edge_GQNN_lf(0.01).pkl Results...
                   0
test_MPE        0.25
test_Sharpness  0.80
test_WS         0.90
------------------------------
GQNN BA_GQNN_lf(0.01).pkl Results...
                   0
test_MPE        0.31
test_Sharpness  0.89
test_WS         0.94
------------------------------
GQNN ER_GQNN_lf(0.01).pkl Results...
                   0
test_MPE 

## Ablation

In [152]:
def summarize_ablation_results(result_dir):
    rows = []

    for fname in os.listdir(result_dir):
        if not fname.endswith('.pkl'):
            continue

        config_name = fname.replace('.pkl', '')
        with open(os.path.join(result_dir, fname), 'rb') as f:
            results = pickle.load(f)

        for run_id, result in results.items():
            row = {'config': config_name, 'run': run_id}

            # 기본 성능 지표
            for k, v in result['test_metrics'].items():
                row[f'test_{k}'] = v

            # loss 항목
            if 'loss_terms' in result:
                for k, v in result['loss_terms'].items():
                    row[f'loss_{k}'] = v

            # 기타
            row['train_time_sec'] = result.get('training_time_sec', None)
            row['gpu_mem_MB'] = result.get('gpu_mem_MB', None)
            row['param_count'] = result.get('param_count', None)

            rows.append(row)

    df = pd.DataFrame(rows)
    return df

# dataset = 'tree'
# result_dir = f"./ablation/{dataset}/"  # 실험 결과 저장 디렉토리
# df_summary = summarize_ablation_results(result_dir)
# df_summary.to_csv(f"./ablation/{dataset}_ablation_summary.csv", index=False)
# print("Ablation summary saved to CSV.")

ablation = 'ablation (0.01)'
dataset = 'basic'

configs_to_include_arch = [
    # Architecture ablation
    "dual_output(1)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(1)",
    "dual_output(1)_fixed_margin(0.05)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(1)",
    "dual_output(1)_fixed_margin(0.10)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(1)",
    "dual_output(0)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(1)",
]

configs_to_include_loss = [
    # Loss ablation
    "dual_output(1)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(1)",
    "dual_output(1)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(0)",
    "dual_output(1)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(0)_use_width_loss(1)",
    "dual_output(1)_fixed_margin(None)_use_sample_loss(0)_use_coverage_loss(1)_use_width_loss(1)",
    "dual_output(1)_fixed_margin(None)_use_sample_loss(0)_use_coverage_loss(0)_use_width_loss(1)",
    "dual_output(1)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(0)_use_width_loss(0)",
]

In [153]:
pd.read_csv(f"./{ablation}/{dataset}_ablation_summary.csv").groupby('config').mean().round(2).loc[configs_to_include_arch].iloc[:, :3]

,run,test_PCIP,test_MPIW
config,,,
dual_output(1)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(1),1.0,0.97,0.87
dual_output(1)_fixed_margin(0.05)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(1),1.0,0.71,0.10
dual_output(1)_fixed_margin(0.10)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(1),1.0,0.86,0.20
dual_output(0)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(1),1.0,0.97,0.85


In [123]:
pd.read_csv(f"./ablation/{dataset}_ablation_summary.csv").groupby('config').mean().round(2).loc[configs_to_include_loss].iloc[:, :3]

,run,test_PCIP,test_MPIW
config,,,
dual_output(1)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(1),2.0,0.91,0.20
dual_output(1)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(0),2.0,1.00,1.20
dual_output(1)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(0)_use_width_loss(1),2.0,0.89,0.17
dual_output(1)_fixed_margin(None)_use_sample_loss(0)_use_coverage_loss(1)_use_width_loss(1),2.0,0.00,0.02
dual_output(1)_fixed_margin(None)_use_sample_loss(0)_use_coverage_loss(0)_use_width_loss(1),2.0,0.00,0.02
dual_output(1)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(0)_use_width_loss(0),2.0,1.00,1.16


In [ ]:
train, test, scalar = True, True, False

folder_path = "/home/sypark/GQNN/ablation/chameleon"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
pkl_files.sort()

for i in range(len(pkl_files)):
    print('Education', pkl_files[i], 'Results...')
    path = os.path.join(folder_path, pkl_files[i])
    # print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    df = summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T
    print(df.round(4))
    print('-' * 30)

### education

In [ ]:
train, test, scalar = False, True, False

folder_path = "/home/sypark/GQNN/ablation/education"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
pkl_files.sort()

for i in range(len(pkl_files)):
    print('Education', pkl_files[i], 'Results...')
    path = os.path.join(folder_path, pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    print('-' * 30)

Education dual_output(0)_use_coverage_loss(0)_use_width_loss(0).pkl Results...
                      0
train_PCIP       0.9995
train_MPIW       0.9768
train_NMPIW      0.9768
train_MCT        0.0972
train_CWC        1.3378
train_MPE        0.2524
train_Sharpness  0.9546
train_WS         0.9768
test_PCIP        0.9920
test_MPIW        0.9764
test_NMPIW       0.7501
test_MCT         0.0898
test_CWC         1.0492
test_MPE         0.2544
test_Sharpness   0.9542
test_WS          0.9770
------------------------------
Education dual_output(0)_use_coverage_loss(0)_use_width_loss(1).pkl Results...
                      0
train_PCIP       0.8973
train_MPIW       0.3179
train_NMPIW      0.3179
train_MCT        0.0012
train_CWC        0.6445
train_MPE        0.0833
train_Sharpness  0.1119
train_WS         0.3221
test_PCIP        0.8997
test_MPIW        0.3197
test_NMPIW       0.2456
test_MCT         0.0014
test_CWC         0.4922
test_MPE         0.0836
test_Sharpness   0.1166
test_WS          0.

### PTBR

In [ ]:
train, test, scalar = True, True, False

folder_path = "/home/sypark/GQNN/ablation/PTBR"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
pkl_files.sort()

for i in range(len(pkl_files)):
    print('Education', pkl_files[i], 'Results...')
    path = os.path.join(folder_path, pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    print('-' * 30)

Education dual_output(0)_use_coverage_loss(0)_use_width_loss(0).pkl Results...
                      0
train_PCIP       0.9992
train_MPIW       1.3476
train_NMPIW      1.3476
train_MCT        0.1338
train_CWC        1.8471
train_MPE        0.2193
train_Sharpness  1.9288
train_WS         1.3476
test_PCIP        0.9796
test_MPIW        1.0717
test_NMPIW       1.7575
test_MCT         0.0847
test_CWC         2.5560
test_MPE         0.1804
test_Sharpness   1.3200
test_WS          1.0729
------------------------------
Education dual_output(0)_use_coverage_loss(0)_use_width_loss(1).pkl Results...
                      0
train_PCIP       0.9467
train_MPIW       0.0802
train_NMPIW      0.0802
train_MCT        0.0044
train_CWC        0.1354
train_MPE        0.0206
train_Sharpness  0.0081
train_WS         0.0806
test_PCIP        0.5749
test_MPIW        0.0725
test_NMPIW       0.1188
test_MCT         0.0199
test_CWC         4.3903
test_MPE         0.0342
test_Sharpness   0.0067
test_WS          0.

### chameleon

In [ ]:
train, test, scalar = False, True, False

folder_path = "/home/sypark/GQNN/ablation/chameleon"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
pkl_files.sort()

for i in range(len(pkl_files)):
    print('Education', pkl_files[i], 'Results...')
    path = os.path.join(folder_path, pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    print('-' * 30)

Education dual_output(0)_use_coverage_loss(0)_use_width_loss(0).pkl Results...
                      0
train_PCIP       0.9997
train_MPIW       0.4051
train_NMPIW      0.4051
train_MCT        0.0404
train_CWC        0.5546
train_MPE        0.1717
train_Sharpness  0.1701
train_WS         0.4051
test_PCIP        0.9824
test_MPIW        0.4020
test_NMPIW       0.6213
test_MCT         0.0335
test_CWC         0.8927
test_MPE         0.1705
test_Sharpness   0.1665
test_WS          0.4031
------------------------------
Education dual_output(0)_use_coverage_loss(0)_use_width_loss(1).pkl Results...
                      0
train_PCIP       0.9105
train_MPIW       0.0099
train_NMPIW      0.0099
train_MCT        0.0003
train_CWC        0.0192
train_MPE        0.0027
train_Sharpness  0.0003
train_WS         0.0100
test_PCIP        0.3873
test_MPIW        0.0090
test_NMPIW       0.0139
test_MCT         0.0045
test_CWC         2.5364
test_MPE         0.0218
test_Sharpness   0.0002
test_WS          0.

### anaheim

In [23]:
train, test, scalar = True, True, False

folder_path = "/home/sypark/GQNN/ablation/Anaheim"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
pkl_files.sort()

for i in range(len(pkl_files)):
    print('Education', pkl_files[i], 'Results...')
    path = os.path.join(folder_path, pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    print('-' * 30)

Education dual_output(0)_use_coverage_loss(0)_use_width_loss(0).pkl Results...
                      0
train_PCIP       0.9995
train_MPIW       1.0857
train_NMPIW      1.0857
train_MCT        0.1080
train_CWC        1.4874
train_MPE        0.3380
train_Sharpness  1.1925
train_WS         1.0857
test_PCIP        1.0000
test_MPIW        0.9648
test_NMPIW       1.2836
test_MCT         0.0965
test_CWC         1.7559
test_MPE         0.2941
test_Sharpness   0.9580
test_WS          0.9648
------------------------------
Education dual_output(0)_use_coverage_loss(0)_use_width_loss(1).pkl Results...
                      0
train_PCIP       0.8809
train_MPIW       0.2651
train_NMPIW      0.2651
train_MCT        0.0051
train_CWC        0.5884
train_MPE        0.0768
train_Sharpness  0.0947
train_WS         0.2689
test_PCIP        0.8516
test_MPIW        0.2881
test_NMPIW       0.3832
test_MCT         0.0140
test_CWC         1.0119
test_MPE         0.0900
test_Sharpness   0.1063
test_WS          0.